### requirements for the following codings


In [5]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.0 MB/s 


### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [2]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings (main dataset)

In [3]:
import numpy as np
import pandas as pd

In [6]:
# training dataset loading
dataset = pd.read_excel('main_ACP_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []

for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('main_ACP_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [7]:
# test dataset loading
dataset = pd.read_excel('main_ACP_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []

for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('main_ACP_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [8]:
# assign the dataset 
X_train_data_name = 'main_ACP_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'main_ACP_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [9]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1378, 320)
(344, 320)
(1378,)
(344,)


### data loading and embeddings (alternative dataset)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# training dataset loading
dataset = pd.read_excel('alternative_ACP_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information

# employ ESM model for converting and save the converted data in csv format
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('alternative_ACP_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

In [ ]:
# test dataset loading
dataset = pd.read_excel('alternative_ACP_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information

# employ ESM model for converting and save the converted data in csv format
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('alternative_ACP_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [ ]:
# assign the dataset 
X_train_data_name = 'alternative_ACP_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'alternative_ACP_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [ ]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### Model architecture

In [16]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### 10-fold cross validation

In [17]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
155/155 [==============================] - 5s 15ms/step - loss: 0.8826 - accuracy: 0.5863 - val_loss: 0.6985 - val_accuracy: 0.5435 - lr: 0.1000
Epoch 2/200
155/155 [==============================] - 2s 14ms/step - loss: 0.6280 - accuracy: 0.6395 - val_loss: 0.6952 - val_accuracy: 0.5435 - lr: 0.1000
Epoch 3/200
155/155 [==============================] - 2s 14ms/step - loss: 0.5803 - accuracy: 0.6855 - val_loss: 0.6364 - val_accuracy: 0.6449 - lr: 0.0600
Epoch 4/200
155/155 [==============================] - 2s 13ms/step - loss: 0.5495 - accuracy: 0.7161 - val_loss: 0.5821 - val_accuracy: 0.6667 - lr: 0.0600
Epoch 5/200
155/155 [==============================] - 2s 13ms/step - loss: 0.5297 - accuracy: 0.7129 - val_loss: 0.6030 - val_accuracy: 0.6232 - lr: 0.0600
Epoch 6/200
155/155 [==============================] - 2s 13ms/step - loss: 0.5002 - accuracy: 0.7306 - val_loss: 0.6234 - val_accuracy: 0.6522 - lr: 0.0360
Epoch 7/200
155/155 [==============================] - 2s 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


155/155 [==============================] - 4s 20ms/step - loss: 1.6469 - accuracy: 0.5016 - val_loss: 0.7585 - val_accuracy: 0.5362 - lr: 0.1000
Epoch 2/200
155/155 [==============================] - 2s 13ms/step - loss: 0.6996 - accuracy: 0.4815 - val_loss: 0.7006 - val_accuracy: 0.4638 - lr: 0.1000
Epoch 3/200
155/155 [==============================] - 2s 14ms/step - loss: 0.6836 - accuracy: 0.5242 - val_loss: 0.6480 - val_accuracy: 0.6159 - lr: 0.0600
Epoch 4/200
155/155 [==============================] - 2s 13ms/step - loss: 0.6643 - accuracy: 0.5815 - val_loss: 0.5993 - val_accuracy: 0.6304 - lr: 0.0600
Epoch 5/200
155/155 [==============================] - 2s 13ms/step - loss: 0.6482 - accuracy: 0.6000 - val_loss: 0.6050 - val_accuracy: 0.6739 - lr: 0.0600
Epoch 6/200
155/155 [==============================] - 2s 13ms/step - loss: 0.6203 - accuracy: 0.6339 - val_loss: 0.6166 - val_accuracy: 0.6667 - lr: 0.0360
Epoch 7/200
155/155 [==============================] - 2s 13ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

155/155 [==============================] - 3s 13ms/step - loss: 1.1691 - accuracy: 0.6177 - val_loss: 0.7089 - val_accuracy: 0.4855 - lr: 0.1000
Epoch 2/200
155/155 [==============================] - 2s 12ms/step - loss: 0.5785 - accuracy: 0.6742 - val_loss: 0.6560 - val_accuracy: 0.5652 - lr: 0.1000
Epoch 3/200
155/155 [==============================] - 2s 12ms/step - loss: 0.5208 - accuracy: 0.7290 - val_loss: 0.6198 - val_accuracy: 0.6304 - lr: 0.0600
Epoch 4/200
155/155 [==============================] - 2s 11ms/step - loss: 0.5113 - accuracy: 0.7484 - val_loss: 0.6277 - val_accuracy: 0.5870 - lr: 0.0600
Epoch 5/200
155/155 [==============================] - 2s 11ms/step - loss: 0.4839 - accuracy: 0.7605 - val_loss: 0.6328 - val_accuracy: 0.6594 - lr: 0.0600
Epoch 6/200
155/155 [==============================] - 2s 11ms/step - loss: 0.4571 - accuracy: 0.7726 - val_loss: 0.6438 - val_accuracy: 0.6884 - lr: 0.0360
Epoch 7/200
155/155 [==============================] - 2s 11ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


156/156 [==============================] - 3s 13ms/step - loss: 0.9254 - accuracy: 0.5254 - val_loss: 0.6949 - val_accuracy: 0.5401 - lr: 0.1000
Epoch 2/200
156/156 [==============================] - 2s 12ms/step - loss: 0.7008 - accuracy: 0.4859 - val_loss: 0.6970 - val_accuracy: 0.4672 - lr: 0.1000
Epoch 3/200
156/156 [==============================] - 2s 11ms/step - loss: 0.6945 - accuracy: 0.5085 - val_loss: 0.7217 - val_accuracy: 0.4599 - lr: 0.0600
Epoch 4/200
156/156 [==============================] - 2s 11ms/step - loss: 0.6957 - accuracy: 0.5165 - val_loss: 0.6927 - val_accuracy: 0.5401 - lr: 0.0600
Epoch 5/200
156/156 [==============================] - 2s 11ms/step - loss: 0.6928 - accuracy: 0.5028 - val_loss: 0.6904 - val_accuracy: 0.5401 - lr: 0.0600
Epoch 6/200
156/156 [==============================] - 2s 12ms/step - loss: 0.6952 - accuracy: 0.4859 - val_loss: 0.6912 - val_accuracy: 0.5401 - lr: 0.0360
Epoch 7/200
156/156 [==============================] - 2s 11ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


156/156 [==============================] - 3s 13ms/step - loss: 0.8293 - accuracy: 0.5761 - val_loss: 0.6957 - val_accuracy: 0.4818 - lr: 0.1000
Epoch 2/200
156/156 [==============================] - 2s 11ms/step - loss: 0.6407 - accuracy: 0.6092 - val_loss: 0.6919 - val_accuracy: 0.5036 - lr: 0.1000
Epoch 3/200
156/156 [==============================] - 2s 12ms/step - loss: 0.6099 - accuracy: 0.6398 - val_loss: 0.5926 - val_accuracy: 0.7080 - lr: 0.0600
Epoch 4/200
156/156 [==============================] - 2s 11ms/step - loss: 0.6088 - accuracy: 0.6471 - val_loss: 0.5819 - val_accuracy: 0.7226 - lr: 0.0600
Epoch 5/200
156/156 [==============================] - 2s 11ms/step - loss: 0.5890 - accuracy: 0.6720 - val_loss: 0.5440 - val_accuracy: 0.7226 - lr: 0.0600
Epoch 6/200
156/156 [==============================] - 2s 12ms/step - loss: 0.5404 - accuracy: 0.6954 - val_loss: 0.5471 - val_accuracy: 0.6934 - lr: 0.0360
Epoch 7/200
156/156 [==============================] - 2s 11ms/step - 

In [18]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.7460753200042315 ± 0.043739056899664536
0.7593259096849925 ± 0.03948824282465656
0.8044658584580148 ± 0.06221850836486582
0.7141859609119703 ± 0.05584179725576553
0.5027567383386884 ± 0.08229734761153824
0.7917416007238911 ± 0.04810856071346626


### model evaluation in test dataset

In [19]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


173/173 [==============================] - 3s 12ms/step - loss: 1.2733 - accuracy: 0.5958 - val_loss: 0.6583 - val_accuracy: 0.6366 - lr: 0.1000
Epoch 2/200
173/173 [==============================] - 2s 12ms/step - loss: 0.5902 - accuracy: 0.6589 - val_loss: 0.6660 - val_accuracy: 0.6221 - lr: 0.1000
Epoch 3/200
173/173 [==============================] - 2s 12ms/step - loss: 0.5379 - accuracy: 0.7163 - val_loss: 0.5802 - val_accuracy: 0.6657 - lr: 0.0600
Epoch 4/200
173/173 [==============================] - 2s 12ms/step - loss: 0.5049 - accuracy: 0.7170 - val_loss: 0.5746 - val_accuracy: 0.6831 - lr: 0.0600
Epoch 5/200
173/173 [==============================] - 2s 11ms/step - loss: 0.5020 - accuracy: 0.7380 - val_loss: 0.5609 - val_accuracy: 0.6919 - lr: 0.0600
Epoch 6/200
173/173 [==============================] - 2s 11ms/step - loss: 0.4607 - accuracy: 0.7583 - val_loss: 0.5553 - val_accuracy: 0.7122 - lr: 0.0360
Epoch 7/200
173/173 [==============================] - 2s 11ms/step - 

In [20]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.7354651162790697
0.7354730757529662
0.7341040462427746
0.7368421052631579
0.4709381919647732
0.8054015684153597


In [22]:
model.save('ACP_main_tensorflow_model',save_format = 'tf') 
!zip -r /content/ACP_main_tensorflow_model.zip /content/ACP_main_tensorflow_model

updating: content/ACP_main_tensorflow_model/ (stored 0%)
updating: content/ACP_main_tensorflow_model/variables/ (stored 0%)
updating: content/ACP_main_tensorflow_model/variables/variables.index (deflated 64%)
updating: content/ACP_main_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 43%)
updating: content/ACP_main_tensorflow_model/saved_model.pb (deflated 88%)
updating: content/ACP_main_tensorflow_model/assets/ (stored 0%)
updating: content/ACP_main_tensorflow_model/keras_metadata.pb (deflated 89%)
